In [ ]:
import os 
import pandas as pd 

from src import config

In [ ]:
!pip install pyspark
!pip install findspark

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

None
None
(112650, 7)
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1

In [ ]:
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
df = pd.read_csv(config.RAW_FILE_PATH / "olist_order_items_dataset.csv")

print(f"""
{df.info()}
{print(df.isna().sum())}
{df.shape}
{df.head()}
""")

In [ ]:
df = df.groupby("order_id").agg({"product_id": lambda x: list(x)})

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1512 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [991 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3231 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2286 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [957 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2799 kB]
Fetched 12.0 MB in 23s (533 kB/s)                                              
Reading package lists... Done
debconf: delaying package configuration, 

In [ ]:
import pyspark.sql.types as T 
import pyspark.sql.functions as F 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

import pyspark.sql.functions as F
from pyspark.ml.fpm import FPGrowth

import findspark
findspark.init()
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pyspark.sql.types as T 
import pyspark.sql.functions as F 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

import pyspark.sql.functions as F
from pyspark.ml.fpm import FPGrowth

import findspark
findspark.init()
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.options(header="true").csv(str(config.RAW_FILE_PATH / "olist_order_items_dataset.csv"))
df = df.dropDuplicates(["order_id", "product_id"]).sort("order_id")

In [ ]:
df = df.groupBy("order_id").agg(F.collect_list("product_id")).sort("order_id")
df.show(2)

+--------------------+------------------------+
|            order_id|collect_list(product_id)|
+--------------------+------------------------+
|00010242fe8c5a6d1...|    [4244733e06e7ecb4...|
|00018f77f2f0320c5...|    [e5f2d52b802189ee...|
+--------------------+------------------------+
only showing top 2 rows



In [ ]:
fpGrowth = FPGrowth(itemsCol='collect_list(product_id)', minSupport=0.00001, minConfidence=0.0001)

model = fpGrowth.fit(df)

In [ ]:
model.freqItemsets.show()
items = model.freqItemsets.toPandas()

model.associationRules.show()
rules = model.associationRules.toPandas()

In [ ]:
items.to_csv(config.INT_FILE_PATH / "mba_freq_items.csv")
rules.to_csv(config.INT_FILE_PATH / "mba_pattern.csv")